In [1]:
import pandas as pd
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from configs import configs
from dataset import ChestXRayCaptionDataset
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from utils import train_transform, evaluate_transform
from model import Chexnet
from pytorch_tokenizer import create_tokenizer
from pytorch_test import evaluation_matrix
from pytorch_chexpert import chexpert
from sklearn.metrics import recall_score, precision_score, f1_score

2091lines [00:00, 149753.09lines/s]
2091lines [00:00, 209685.12lines/s]


caller: c:\Users\darkenstardragon\Documents\Work\chest-xray-report-gen\text_generation\pytorch_chexpert.py
Creating Chexpert reward module...
Using 1 GPUs!


In [2]:
mean_path = 'weights/pca_mean_full.npy'
components_path = 'weights/pca_components_full.npy'
LOAD_RANDOM_PROJECTION_DATA = True
RANDOM_PROJECT = True
RANDOM_PROJECT_DIM = 1024
SEED = 0

filename = {
    'train_x': 'baseline_data/' + configs['mimic_dir'] + f'train_image_embeddings_{RANDOM_PROJECT_DIM}_{SEED}.npy',
    'train_y': 'baseline_data/' + configs['mimic_dir'] + f'train_captions_{RANDOM_PROJECT_DIM}_{SEED}.npy',
    'val_x': 'baseline_data/' + configs['mimic_dir'] + f'val_image_embeddings_{RANDOM_PROJECT_DIM}_{SEED}.npy',
    'val_y': 'baseline_data/' + configs['mimic_dir'] + f'val_captions_{RANDOM_PROJECT_DIM}_{SEED}.npy',
    'test_x': 'baseline_data/' + configs['mimic_dir'] + f'test_image_embeddings_{RANDOM_PROJECT_DIM}_{SEED}.npy',
    'test_y': 'baseline_data/' + configs['mimic_dir'] + f'test_captions_{RANDOM_PROJECT_DIM}_{SEED}.npy',
}

In [3]:
tokenizer = create_tokenizer()
encoder = Chexnet.finetuned()
pca_mean = None
pca_components = None
train_loader = DataLoader(
    ChestXRayCaptionDataset('train', transform=train_transform),
    batch_size=8,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
)

val_loader = DataLoader(
    ChestXRayCaptionDataset('val', transform=evaluate_transform),
    batch_size=8,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
)

test_loader = DataLoader(
    ChestXRayCaptionDataset('test', transform=evaluate_transform),
    batch_size=8,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
)

2091lines [00:00, 190608.75lines/s]


loading weights from weights/pretrained_encoder/pretrained_enc_epoch_5_2022-03-08_15-43-47.540586.pth.tar


In [4]:
def generate_image_embeddings(encoder, data_loader):
    encoder.eval()
    image_embeddings = []
    probs = []
    captions = []

    with torch.no_grad():
        for i, (img, caption, _) in enumerate(tqdm(data_loader)):
            img = img.cuda()
            encoded_img, prob = encoder(img)
            image_embeddings.append(encoded_img.cpu())
            probs.append(prob.cpu())
            captions.append(caption.cpu())

    image_embeddings = torch.cat(image_embeddings).reshape(-1, 1024*8*8).numpy()
    probs = torch.cat(probs).numpy()
    captions = torch.cat(captions).numpy()
    return image_embeddings, probs, captions

def generate_image_embeddings_random(encoder, data_loader, projection_matrix, project_every=2):
    # With random projection
    encoder.eval()
    image_embeddings = []
    captions = []
    batch = []
    with torch.no_grad():
        for i, (img, caption, _) in enumerate(tqdm(data_loader)):
            img = img.cuda()
            encoded_img, prob = encoder(img)
            batch.append(encoded_img.cpu())
            captions.append(caption.cpu())
            if ((i+1) % project_every) == 0 or (i+1) == len(data_loader):
                batch = torch.cat(batch).reshape(-1, 1024*8*8).numpy()
                batch = np.matmul(batch, projection_matrix)
                image_embeddings.append(batch)
                batch = []

    image_embeddings = np.vstack(image_embeddings)
    captions = torch.cat(captions).numpy()
    return image_embeddings, captions

def load_pca():
    global pca_mean, pca_components
    pca_mean = np.load(mean_path)
    pca_components = np.load(components_path)

def pca_transform(embeddings):
    if type(pca_mean) == type(None) or type(pca_components) == type(None):
        load_pca()
    return np.dot(embeddings - pca_mean, pca_components.T)

In [5]:
if RANDOM_PROJECT:
    if LOAD_RANDOM_PROJECTION_DATA:
        train_image_embeddings = np.load(filename['train_x'])
        train_captions = np.load(filename['train_y'])
        print(train_image_embeddings.shape)
        print(train_captions.shape)
    else:
        rng = np.random.RandomState(SEED)
        # Gaussian random projection
        projection_matrix = rng.normal(0.0, 1/RANDOM_PROJECT_DIM, (65536, RANDOM_PROJECT_DIM))
        project_every = 256
        train_image_embeddings, train_captions = generate_image_embeddings_random(encoder, train_loader, projection_matrix, project_every=project_every)
        print(train_image_embeddings.shape)
        print(train_captions.shape)
        np.save(filename['train_x'], train_image_embeddings)
        np.save(filename['train_y'], train_captions)
else:
    train_image_embeddings, train_probs, train_captions = generate_image_embeddings(encoder, train_loader)


(205181, 1024)
(205181, 246)


In [6]:
# pca training
# if not RANDOM_PROJECT:
#     pca = PCA(n_components=1024)
#     pca.fit(train_image_embeddings)
#     np.save(mean_path, pca.mean_)
#     np.save(components_path, pca.components_)

In [7]:
if RANDOM_PROJECT:
    if LOAD_RANDOM_PROJECTION_DATA:
        val_image_embeddings = np.load(filename['val_x'])
        val_captions = np.load(filename['val_y'])
        test_image_embeddings = np.load(filename['test_x'])
        test_captions = np.load(filename['test_y'])
        print(val_image_embeddings.shape)
        print(val_captions.shape)
        print(test_image_embeddings.shape)
        print(test_captions.shape)
    else:
        rng = np.random.RandomState(SEED)
        # Gaussian random projection
        projection_matrix = rng.normal(0.0, 1/RANDOM_PROJECT_DIM, (65536, RANDOM_PROJECT_DIM))
        project_every = 256
        val_image_embeddings, val_captions = generate_image_embeddings_random(encoder, val_loader, projection_matrix, project_every=project_every)
        print(val_image_embeddings.shape)
        print(val_captions.shape)
        test_image_embeddings, test_captions = generate_image_embeddings_random(encoder, test_loader, projection_matrix, project_every=project_every)
        print(test_image_embeddings.shape)
        print(test_captions.shape)
        np.save(filename['val_x'], val_image_embeddings)
        np.save(filename['val_y'], val_captions)
        np.save(filename['test_x'], test_image_embeddings)
        np.save(filename['test_y'], test_captions)
else:
    val_image_embeddings, val_probs, val_captions = generate_image_embeddings(encoder, val_loader)
    test_image_embeddings, test_probs, test_captions = generate_image_embeddings(encoder, test_loader)

(27358, 1024)
(27358, 246)
(41037, 1024)
(41037, 246)


In [8]:
if not RANDOM_PROJECT:
    train_image_embeddings = pca_transform(train_image_embeddings)
    val_image_embeddings = pca_transform(val_image_embeddings)
    test_image_embeddings = pca_transform(test_image_embeddings)

In [9]:
# Fit pca transformed into knn
indices = [*range(train_image_embeddings.shape[0])]
one_nn = KNeighborsClassifier(n_neighbors=1)
one_nn.fit(train_image_embeddings, indices)

KNeighborsClassifier(n_neighbors=1)

In [10]:
def predict(embeddings, decode=False, batch_size=64):
    captions = []
    data_loader = DataLoader(
        embeddings,
        batch_size=batch_size,
        num_workers=0,
        pin_memory=True,
    )
    for j, batch in enumerate(tqdm(data_loader)):
        dists, indices = one_nn.kneighbors(batch)
        captions.extend([train_captions[i] for i in indices])
    captions = np.array(captions).reshape(embeddings.shape[0], -1)
    if decode:
        captions = tokenizer.decode(captions)
    return captions

def evaluate(true_captions, pred_captions, batch_size):
    true_df = []
    pred_df = []

    true_loader = DataLoader(
        true_captions, 
        batch_size=batch_size,
        num_workers=0,
        pin_memory=True
    )
    pred_loader = DataLoader(
        pred_captions, 
        batch_size=batch_size,
        num_workers=0,
        pin_memory=True
    )

    for t in tqdm(true_loader):
        labels = chexpert(t, tokenizer)
        true_df.append(labels)

    for p in tqdm(pred_loader):
        labels = chexpert(p, tokenizer)
        pred_df.append(labels)
    
    true_df = pd.concat(true_df).reset_index(drop=True)
    pred_df = pd.concat(pred_df).reset_index(drop=True)
    return evaluation_matrix(true_df, pred_df)

In [10]:
predicted_reports = predict(val_image_embeddings, batch_size=1024)
val_eval_matrix = evaluate(val_captions, predicted_reports, batch_size=12)

100%|██████████| 2280/2280 [05:25<00:00,  7.01it/s]


In [11]:
val_eval_matrix.round(4)

Metrics,Recall,Precision,F1
Enlarged Cardiomediastinum,0.7247,0.6553,0.6883
Cardiomegaly,0.6668,0.5996,0.6314
Lung Opacity,0.7508,0.6905,0.7194
Lung Lesion,0.2947,0.3424,0.3167
Edema,0.6026,0.5572,0.5790
Consolidation,0.6255,0.6054,0.6153
Pneumonia,0.4418,0.4656,0.4534
Atelectasis,0.6458,0.5931,0.6183
Pneumothorax,0.2180,0.2429,0.2298
Pleural Effusion,0.6297,0.5805,0.6041


In [11]:
predicted_reports = predict(test_image_embeddings)
test_eval_matrix = evaluate(test_captions, predicted_reports, batch_size=12)

100%|██████████| 642/642 [08:14<00:00,  1.30it/s]


TypeError: evaluate() missing 1 required positional argument: 'batch_size'

In [12]:
test_eval_matrix = evaluate(test_captions, predicted_reports, batch_size=12)

100%|██████████| 3420/3420 [08:23<00:00,  6.79it/s]


In [13]:
test_eval_matrix.round(4)

Metrics,Recall,Precision,F1
Enlarged Cardiomediastinum,0.7283,0.6613,0.6932
Cardiomegaly,0.6675,0.6090,0.6369
Lung Opacity,0.7548,0.6915,0.7218
Lung Lesion,0.3034,0.3461,0.3234
Edema,0.6008,0.5565,0.5778
Consolidation,0.6302,0.6059,0.6178
Pneumonia,0.4529,0.4659,0.4593
Atelectasis,0.6590,0.6060,0.6314
Pneumothorax,0.2444,0.2675,0.2555
Pleural Effusion,0.6448,0.6047,0.6241
